# A tips on how to get and preprocess the shared datasets 

    # Collection and Curation ... 

In [1]:
import numpy as np
import pandas as pd
import preprocessor as p
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer
from nltk import FreqDist
import codecs, json, string, re, time, os, sys, datetime
import tweepy

#### SOCIAL STREAM CATEGORY I

    #Data from Twitter social networks (Twitter) ... using specific usernames and keywords for data collection 
    
    The main collection sources are news channels (from respective websites and corresponding social media handles). This collection mostly consists of well-written text (abidng by formal writing style) to post or break news on social media platforms such as Twitter. The category consist of diverse users that interacts with the post from News Channel. The users' content and their relevant networks are used to collect further data for the analysis. 

In [3]:
#THE FOLLOWING KEYS NEED TO BE OBTAINED FROM TWITTER TO ACCESS THE API .... 
access_token = ("YOUR ACCESS TOKEN HERE")
access_token_secret = ("YOUR ACCESS TOKEN SECRET HERE")
consumer_key = ("YOUR CONSUMER KEY HERE")
consumer_secret = ("YOUR CONSUMER SECRET KEY HERE")
# authentication instance:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) # required for validation
auth.set_access_token(access_token, access_token_secret)#..access after validation of user
api =  tweepy.API(auth) # tweepy requires authenticated user to operate
#HAUSE-BASED TWITTER ACCOUNTS TO QUERY FOR COLLECTION: 
#users = ['list of users to retrieve tweets from']

    # THE COLLECTION: 

In [ ]:
#save all extracts as dataframes: 
extracts = {'Date':[],'TweetSource':[],'MTAccountID':[],'MainTweet':[],'MainTweetID':[],\
            'MTRTCount':[],'MTweetFVCount':[],'MTFollowerCount':[],'MTFriendsCount':[],\
            'MainACreated':[], 'MTLanguage':[],'RepTAccountID':[],'RepACreated':[],\
            'MTReply':[], 'ReplierName':[],'RepTweetFVCount':[],'RepTRTCount':[],\
            'RepTFollowerCount':[], 'RepTFriendsCount':[],'RepTLanguage':[]}  
start = time.time()

for user in users:
    print(f'processing {user} account')    
    
    try:
        mt_timeline=tweepy.Cursor(api.user_timeline,screen_name=user).items(21)
        for main_item in mt_timeline:

            main_user = main_item.user # info/metadata about the user
            main_tweet = main_item # tweet object with many extractable fields
            
            # EXTRACT THE REPLIES FROM THE MAIN TWEET VIA SEARCH: 
            mt_replies=tweepy.Cursor(api.search,q='to:'+user,result_type='recent').items(101)#(203)
            for rep_tweet in mt_replies: #for each reply tweet, document the following: 
                if hasattr(rep_tweet, 'in_reply_to_status_id_str'):
                    if (rep_tweet.in_reply_to_status_id_str == main_tweet.id_str):
                        #print(f'updating {user} replies ...')
                        
                        #UPDATE EXTRACTS: MAIN TWEET 
                        extracts['Date'].append(main_tweet.created_at)
                        extracts['TweetSource'].append(user[1:].upper())
                        extracts['MTAccountID'].append(main_item.user.id)
                        extracts['MainTweet'].append(main_tweet.text)
                        extracts['MainTweetID'].append(main_item.id)
                        extracts['MTRTCount'].append(main_item.retweet_count)
                        extracts['MTFollowerCount'].append(main_item.user.followers_count)
                        extracts['MTFriendsCount'].append(main_item.user.friends_count)
                        extracts['MTweetFVCount'].append(main_item.user.favourites_count)
                        extracts['MainACreated'].append(main_item.user.created_at)
                        extracts['MTLanguage'].append(main_item.lang)

                        #UPDATE EXTRACTS: REPLYING TWEET:
                        extracts['RepTAccountID'].append(rep_tweet.user.id)
                        extracts['RepACreated'].append(rep_tweet.user.created_at)
                        extracts['MTReply'].append(' '.join(rep_tweet.text.split()[1:]))
                        extracts['ReplierName'].append(rep_tweet.user.screen_name)
                        extracts['RepTweetFVCount'].append(rep_tweet.user.favourites_count)
                        extracts['RepTRTCount'].append(rep_tweet.retweet_count)
                        extracts['RepTFollowerCount'].append(rep_tweet.user.followers_count)
                        extracts['RepTFriendsCount'].append(rep_tweet.user.friends_count)
                        extracts['RepTLanguage'].append(rep_tweet.lang)
        
        df_temp = pd.DataFrame(extracts)
        
        #save output: 
        with open('~/miscellaneous-hausa-corpus1.csv','a') as csv:
            df_temp.to_csv(csv, header=False, mode='a',index_label=False)
    
    except tweepy.TweepError:
        time.sleep(20*5)
        continue
    except StopIteration:
        break

stop = time.time()-start
print('The process took: ',stop/60, ' minutes')

In [7]:
df = pd.DataFrame(extracts)
df.to_csv('~/miscellaneous-hausa-corpus1.csv')
df.head(1)

,Date,TweetSource,MTAccountID,MainTweet,MainTweetID,MTRTCount,MTweetFVCount,MTFollowerCount,MTFriendsCount,MainACreated,MTLanguage,RepTAccountID,RepACreated,MTReply,ReplierName,RepTweetFVCount,RepTRTCount,RepTFollowerCount,RepTFriendsCount,RepTLanguage
0,2020-12-25 01:14:25,BBCHAUSA,18168536,Jurgen Klopp ya lashe kyautar Fifa ta gwarzon ...,1342277487536066562,4,112,806789,49,2008-12-16 18:29:17,in,1147282749981691907,2019-07-05 23:14:57,Abunda ya faru tun last week,nasshameed,3025,0,105,404,in


    # Data Sharing ...  
For sentence generation, each main tweet (tweet tweet that atracts/generates replies from other user), is  followed by the corresponding reply as paragraph (n sentences per line). The final data is saved as a text file (*.txt). Each main tweet begins with the prefix MT and the corresponding reply begins with MTR (i.e. main tweet reply). 

The only preprocessing performed on the data is to remove urls and user mentions  Because some of the replies contain only user mention or emojis, the cleaned version of the data often contain/returne empty space for the MTR, thus, both the raw and cleaned version of the datasets are returned (using the suffix raw and cleaned, respectively). 

In [16]:
df = pd.read_csv('data/social-media/twitter/corpus1/tweets_corpus1-for-sharing.csv')
df = df[['MainTweetID', 'MainTweet', 'MTReply','Date', 'TweetSource']]
df.head(4)

,MainTweetID,MainTweet,MTReply,Date,TweetSource
8,1342444287359909888,Christmas 2020: Tsaron ƴan Najeriya zai zama w...,https://t.co/xNZBHzuoCd,2020-12-25 12:17:14,BBCHAUSA
9,1342441579022266369,"Korona ta yi ajalin jagoran ƴan adawan Mali, S...",Too Allah ya kyauta,2020-12-25 12:06:28,BBCHAUSA
10,1342441579022266369,"Korona ta yi ajalin jagoran ƴan adawan Mali, S...",Allah yajikansa yasa aljannace makomarsa ameen...,2020-12-25 12:06:28,BBCHAUSA
11,1342441579022266369,"Korona ta yi ajalin jagoran ƴan adawan Mali, S...",Ayya Allah ya gafarta masa.,2020-12-25 12:06:28,BBCHAUSA


**PARALLEL DATASET**

In [20]:
ps=pd.read_csv('data/social-media/twitter/parallel-data/parallel-hausa-tweets.csv')
ps.head()

,CleanedMainT,CleanedReplyT,Hausa2EngMainT,Hausa2EngReplyT
0,Jurgen Klopp ya lashe kyautar Fifa ta gwarzon ...,Abunda ya faru tun last week,Jurgen Klopp has won the Fifa World Coach of t...,It's been a while since last week
1,Jurgen Klopp ya lashe kyautar Fifa ta gwarzon ...,Ku sai yanzu ku ke sawa news,Jurgen Klopp has won the Fifa World Coach of t...,You are now wearing news
3,Ƴan bindiga sun kashe mutum da ƙona buhunan ma...,Allah ya isa tsakaninmu da duk wanda yake da s...,Gunmen kill man and burn sacks of maize in Kaduna,May Allah reach between us and all those who a...
4,Ƴan bindiga sun kashe mutum da ƙona buhunan ma...,Innalillahi wa innailaihi rajiun. Wannan kasar...,Gunmen kill man and burn sacks of maize in Kaduna,Innalillahi wa innailaihi rajiun. This country...
5,Ƴan bindiga sun kashe mutum da ƙona buhunan ma...,Subhnallah,Gunmen kill man and burn sacks of maize in Kaduna,Subhnallah


**WEB DATA Category:**

    #Web Scrapping using BeutifulSoup package to scrap useful data from relevan websites that provide information in Hausa language

In [17]:
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import re

**Web data ...**

In [5]:
dfs = pd.read_csv('/home/ijdutse/analysis/hausa-corpus/data/web-sites/misc-collections-2021-01-05 20:35:14.csv')
dfs.head(3)

,NewsText,Theme,Source
1,Ƴan sandan sun ce an kai samamen ne a unguwann...,News2021-01-05,WWW.BBC
2,Mai magana da yawun ƴan sanda Olumuyiwa Adejob...,News2021-01-05,WWW.BBC
3,Hukumomi sun ce za su tsaurara kai samamen don...,News2021-01-05,WWW.BBC


**PDF WEB FILES ...**

In [2]:
dweb =pd.read_csv('/home/ijdutse/analysis/hausa-corpus/data/4share/pdf-web-files.csv')
dweb.tail()

,Report,Source
1,"John Bray, ya yi gayyar sanya hannun sayar da ...",KasarAmirkaTaYiGayyarSanyaHannunKamfaninAirPea...
2,"Babban Jami’in Jakadancin, Bray ne ya shaida s...",KasarAmirkaTaYiGayyarSanyaHannunKamfaninAirPea...
3,"A jawabinsa na marhabin, babban jami’in jakada...",KasarAmirkaTaYiGayyarSanyaHannunKamfaninAirPea...
4,"“Gwamnatin {asar Amirka, ta dage, wajen fa]a]a...",KasarAmirkaTaYiGayyarSanyaHannunKamfaninAirPea...


In [93]:
###########################################################################################